# Non linear analysis - Assignment 3

18.10.2023 - Jérémie Engler and Laure Toullier

In [2]:
#imports

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('bmh')
import math
import pandas as pd

np.set_printoptions(precision=5, suppress=True, linewidth=150)

Params = plt.rcParams
Params['figure.figsize'] = (14, 7) 

General data

In [110]:
E = 200e3 #Mpa
poisson = 0
sigma_y = 400 #Mpa

eps_0 = np.zeros((2,))
sigma_0 = np.zeros((2,))

strain_incr = np.array([0.003,0.004])

d = E/(1 - poisson**2) #Factor of the constitutive matrix

D = np.full((2,2), poisson)
np.fill_diagonal(D, 1)
D = D * d


(2, 2)


In [141]:
def stress_incr_computation(strain_incr, plastic_strain_incr, n, D):
    if plastic_strain_incr.all() == 0:
        stress_incr = D @ strain_incr
    else:
        stress_incr = (D - (D @ n.T @ n @ D) / (n @ D @ n.T)) @ plastic_strain_incr
        correction_factor = (D @ n.T @ n @ D) / (n @ D @ n.T)
        trial_factor = D

    return stress_incr

def check_failure(stress):
    fail = False
    criterion = stress[0]**2 + stress[1]**2 - stress[0] * stress[1] - sigma_y ** 2

    if criterion > 0:
        fail = True

    return fail

def failure_value(stress):
    criterion = stress[0]**2 + stress[1]**2 - stress[0] * stress[1] - sigma_y ** 2

    return criterion

def strain_decomposition(strain_incr, alpha):
    elastic_strain_incr = alpha * strain_incr
    plastic_strain_incr = (1 - alpha) * strain_incr

    return elastic_strain_incr, plastic_strain_incr

def norm_criterion_surface(stress_y):
    n1 = 2 * stress_y[0] - stress_y[1]
    n2 = 2 * stress_y[1] - stress_y[0]
    n = np.array([[n1,n2]])
    return n

In [146]:
# Stress increment from strain increment

plastic_strain_incr = np.zeros((2,))
n = np.zeros((2,))

# First forward step

stress_incr = stress_incr_computation(strain_incr, plastic_strain_incr, n, D)

stress_0 = sigma_0 + stress_incr

print(f"Stress at first step: {stress_0} Mpa ")

if check_failure(stress_0) == True:
    alpha = np.sqrt((sigma_y**2) / (stress_0[0]**2 + stress_0[1]**2 - stress_0[0] * stress_0[1]))
    stress_y = alpha * stress_0

print(f"Criterion value at first step = {failure_value(stress_y)}")
print(f"Yielding stress at first step: {stress_y} Mpa \n")

elastic_strain_incr, plastic_strain_incr = strain_decomposition(strain_incr, alpha)


# Second forward step

sub_increments = 2

plastic_strain_incr_2 = plastic_strain_incr / sub_increments

print(f"Plastic strain increment = {plastic_strain_incr}")
print(f"Plastic strain sub_increment = {plastic_strain_incr_2} \n")

stress = stress_y

for sub_increment in range(sub_increments):
    n = norm_criterion_surface(stress)
    print(f'n{sub_increment + 1} = {n}')

    stress_incr = stress_incr_computation(strain_incr, plastic_strain_incr_2, n, D)
    print(f"Δσ{sub_increment + 1} = {stress_incr}")

    stress = stress + stress_incr
    
    print(f"Stress, sub-increment {sub_increment + 1}: {stress} Mpa \n")

print(f"Stress with {sub_increments} sub-increments: {stress} Mpa")
print(f"Criterion value = {failure_value(stress)}")


Stress at first step: [600. 800.] Mpa 
Criterion value at first step = 2.9103830456733704e-11
Yielding stress at first step: [332.82012 443.76016] Mpa 

Plastic strain increment = [0.00134 0.00178]
Plastic strain sub_increment = [0.00067 0.00089] 

n1 = [[221.88008 554.7002 ]]
Δσ1 = [ 53.74308 -21.49723]
Stress, sub-increment 1: [386.5632  422.26293] Mpa 

n2 = [[350.86347 457.96265]]
Δσ2 = [-1.81153  1.38788]
Stress, sub-increment 2: [384.75167 423.65081] Mpa 

Stress with 2 sub-increments: [384.75167 423.65081] Mpa
Criterion value = 4513.499097270338


In [ ]:
def forward_euler():

    plastic_strain_incr = np.zeros((2,))
    n = np.zeros((2,))

    # First forward step

    stress_incr = stress_incr_computation(strain_incr, plastic_strain_incr, n, D)

    stress_0 = sigma_0 + stress_incr

    print(f"Stress at first step: {stress_0} Mpa ")

    if check_failure(stress_0) == True:
        alpha = np.sqrt((sigma_y**2) / (stress_0[0]**2 + stress_0[1]**2 - stress_0[0] * stress_0[1]))
        stress_y = alpha * stress_0

    print(f"Criterion value at first step = {failure_value(stress_y)}")
    print(f"Yielding stress at first step: {stress_y} Mpa \n")

    elastic_strain_incr, plastic_strain_incr = strain_decomposition(strain_incr, alpha)


    # Second forward step

    sub_increments = 2

    plastic_strain_incr_2 = plastic_strain_incr / sub_increments

    print(f"Plastic strain increment = {plastic_strain_incr}")
    print(f"Plastic strain sub_increment = {plastic_strain_incr_2} \n")

    stress = stress_y

    for sub_increment in range(sub_increments):
        n = norm_criterion_surface(stress)
        print(f'n{sub_increment + 1} = {n}')

        stress_incr = stress_incr_computation(strain_incr, plastic_strain_incr_2, n, D)
        print(f"Δσ{sub_increment + 1} = {stress_incr}")

        stress = stress + stress_incr
        
        print(f"Stress, sub-increment {sub_increment + 1}: {stress} Mpa \n")

    print(f"Stress with {sub_increments} sub-increments: {stress} Mpa")
    print(f"Criterion value = {failure_value(stress)}")

    return None
